In [4]:
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
!pip install autocorrect
from autocorrect import Speller
from collections import OrderedDict 
from pathlib import Path
import shutil
import spacy 
import re
nltk.download('wordnet')
nltk.download('stopwords')
!pip install contractions
!pip install spacy
!pip install textblob
import contractions
from textblob import TextBlob, Word
import pandas as pd
import numpy as np
import ast

     |████████████████████████████████| 624kB 5.5MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.4.0-cp37-none-any.whl size=621775 sha256=b8d1b0e9b9147fde9c58816123adf001bd96f21d3481c98fe026c485cbc42c80
  Stored in directory: /root/.cache/pip/wheels/1d/b0/d4/b941891ad0f8d8847be03583e21e68ed4732d763c71a6c0943
Successfully built autocorrect
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 327kB 5.3MB/s 
     |████████████████████████████████| 266kB 11.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85400 sha256=25b3c14614544b389e3c177cc379799d1d95ee15a6f8894c9d940d996a7f99bf
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
paths = []
filename = []
for (dirpath, dirnames, filenames) in os.walk("/content/drive/MyDrive/Classroom/CSE508: Information Retrieval/Assignment 2/stories"):
  for i in filenames:
    paths.append(str(dirpath)+str("/")+i)
    filename.append(i)

In [5]:
#Rahul's cell
paths = []
filename = []
# /content/drive/MyDrive/stories
# /content/drive/MyDrive/IR ASSIGNMENT 2/stories
for (dirpath, dirnames, filenames) in os.walk("/content/drive/MyDrive/stories"):
  for i in filenames:
    paths.append(str(dirpath)+str("/")+i)
    filename.append(i)

In [2]:
documentsDict = {}
# filenameOfDoc = {}
# global counter 
# counter = 0
def cleaning(filepath,filename,flag):
  if(flag==1):
    f = open(filepath, 'r', errors = 'ignore',encoding='cp1250').read().strip()
    clean_S = contractions.fix(f)
    clean_S=re.sub("([\-]{1,2})", " ",clean_S)
    clean_s=re.sub("[\,]", " ",clean_S)
    clean_S = re.sub("[\.]"," ",clean_S)
    clean_S = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", clean_S)  #junk symbols
    # clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S=re.sub("([\.\!]{2})", " ",clean_S)  #removing punctuations
    clean_S = re.sub('\s+', ' ', clean_S)    
    clean_S=re.sub("[^a-zA-Z\s\n]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
    clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S = clean_S.lower()
    documentsDict[filename]=clean_S
    # counter = counter+1
    # filenameOfDoc[counter] = filename

  elif(flag==0):
    filepath = contractions.fix(filepath)
    filepath=re.sub("([\-]{1,2})", " ",filepath)
    filepath=re.sub("[0-9]", "",filepath)
    # filepath=re.sub("[\,]", " ",filepath)
    filepath = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", filepath)
    # filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath=re.sub("([\.\!]{2})", " ",filepath)
    filepath = re.sub('\s+', ' ', filepath)
    filepath=re.sub("[^a-zA-Z\s\n]", " ",filepath)
    filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath = filepath.lower()
    return filepath

In [6]:
for i in range(0,len(filename)):
       cleaning(paths[i],filename[i],1)

In [8]:
len(documentsDict)

467

In [23]:
f = open('cleanedForJaccard.txt', 'w')
f.write(str(documentsDict))
f.close()

In [29]:
f = open('cleanedForJaccard.txt', 'r')
documentsDict = f.read()
documentsDict = ast.literal_eval(documentsDict)
f.close()
documentsFilename = list(documentsDict.keys())

In [9]:
def gen_token(flag,query):
  tokenizer = RegexpTokenizer(r'\w+')
  if(flag==1):
    for docname in documentsDict:
        documentsDict[docname] = tokenizer.tokenize(str(documentsDict[docname]))

  elif(flag==0):
    query = tokenizer.tokenize(query)
    return query

def remove_stopword(flag,query):
  stopwordsList = stopwords.words('english')
  if(flag==1):
    for docname in documentsDict:
        for word in documentsDict[docname]:
            if word in stopwordsList:
                documentsDict[docname].remove(word)
  elif(flag==0):
      q=[]
      for word in query:
            if word not in stopwordsList:
              q.append(word)
      query=q.copy()
      return query

def normalise_using_textblob(flag,query):
  if(flag==1):       
    for docname in documentsDict:
        dataList = []
        for word in documentsDict[docname]:
            w = Word(word)
            dataList.append(w.lemmatize())
        documentsDict[docname] = dataList   

  elif(flag==0):
    ans = []
    for word in query:
      w=Word(word)
      ans.append(w.lemmatize())
    return ans

In [10]:
def proquery(query):
  queryFinal=[]
  query = cleaning(query," ",0)
  queryFinal = gen_token(0,query)
  queryFinal = remove_stopword(0,queryFinal)
  queryFinal = normalise_using_textblob(0,queryFinal)
  #queryFinal = queryFinal.replace(","," ").split()
  return queryFinal

In [11]:
gen_token( 1 , " " )
remove_stopword(1,"")
normalise_using_textblob(1,"")

In [18]:

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

document_list = []
document_name_list = []
def jaccard_on_documents(documentsDict,query):
  jaccard_similar_list = []

  for i in documentsDict.keys():
    document_name_list.append(i)
    document_list.append(documentsDict[i])
    jaccard_similar_list.append(jaccard_similarity(documentsDict[i],query))

  top_jaccard_pos = []

  max1 = 0
  pos1 = 0

  for i in range(5):
    max1 = 0
    pos1 = 0
    for j in range(0,len(jaccard_similar_list)):
      flag = 0
      for k in range(len(top_jaccard_pos)):
        if j==top_jaccard_pos[k]:
          flag = 1
      if flag == 0:
        temp = jaccard_similar_list[j]
        if(temp>max1):
          max1 = temp
          pos1 = j
    max1
    pos1
    top_jaccard_pos.append(pos1)
  return top_jaccard_pos





In [30]:
query = "THE CRAB AND THE HERON"
tokenquery = proquery(query)
top_jaccard_pos = jaccard_on_documents(documentsDict,tokenquery)
# print(top_jaccard_pos)
print("Entered Query: "+ str(tokenquery))
print("Top 5 Documents Retrieved")
for i in top_jaccard_pos:
  # print(i)
  # print(document_list[i])
  print(document_name_list[i])

Entered Query: ['crab', 'heron']
Top 5 Documents Retrieved
crabhern.txt
aesopa10.txt
long1-3.txt
aesop11.txt
fgoose.txt


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Cosine part in Q2_TFIDF file

In [ ]:
#COSINE MOVOED TO ANOTHER FILE

In [ ]:
#I need these two things to show Cosine similarity and find top 5 documents
documentsDict
list2 = ['the', 'vigilante', 'hello', 'greetings', 'salutations']

In [ ]:
document_list = []
def cosine_on_documents(documentsDict,query):
  cosine_similar_list = []

  for i in documentsDict.keys():
    document_list.append(documentsDict[i])

    X_set = set()
    Y_set = set()
    for j in documentsDict[i]:
      X_set.add(j)
    for j in query:
      Y_set.add(j)

    rvector = X_set.union(Y_set) 

    l1 =[]
    l2 =[]
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0
      
    # cosine formula 
    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    cosine_similar_list.append(cosine)
  return cosine_similar_list

cosine_similar_list = cosine_on_documents(documentsDict,list2)
cosine_similar_list

[0.017650112740455196,
 0.015861031714362882,
 0.042835293687811936,
 0.019706585563285865,
 0.016043698370396316,
 0.03907323325822818,
 0.006442903104588024,
 0.018569533817705187,
 0.012524485821702989,
 0.01771924779845835,
 0.02654893246399888,
 0.01405109839972998,
 0.02507849312877596,
 0.0158810204201071,
 0.017460757394239454,
 0.035245368842512066,
 0.016552117772047358,
 0.017650112740455196,
 0.028912724941310364,
 0.02082881368183567,
 0.030015011259383207,
 0.011278574072347185,
 0.015343813327075089,
 0.007765014339568158,
 0.024701609087778548,
 0.0173421993904824,
 0.022587697572631283,
 0.015168500428005318,
 0.020327890704543543,
 0.028973338686962963,
 0.007220640094792102,
 0.023973165074269207,
 0.021952851997938068,
 0.01409289650415634,
 0.019480930197663147,
 0.034001020045902296,
 0.03793216209054408,
 0.023735633163877067,
 0.018077538151554683,
 0.018601633295108114,
 0.04624972900628803,
 0.031008683647302117,
 0.022167554926073633,
 0.03154401489382559,
 0

In [ ]:
document_list = [[1,2,3,4],[5,6,7,8]]
def cosine_on_documents(documentsList,query):
  cosine_similar_list = []
  for i in documentsList:
    l1 = i
    l2 = query
    # for w in rvector:
    #     if w in X_set: l1.append(1) # create a vector
    #     else: l1.append(0)
    #     if w in Y_set: l2.append(1)
    #     else: l2.append(0)
    c = 0
    
    for i in range(len(l1)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    cosine_similar_list.append(cosine)
  return cosine_similar_list

cosine_similar_list = cosine_on_documents(document_list,[1,2,3,4])
cosine_similar_list

[3.0, 4.341215710622296]

In [ ]:
def topDocuments(cosine_similar_list):
  top_cosine_pos = []

  max1 = 0
  pos1 = 0

  for i in range(5):
    max1 = 0
    pos1 = 0
    for j in range(0,len(cosine_similar_list)):
      flag = 0
      for k in range(len(top_cosine_pos)):
        if j==top_cosine_pos[k]:
          flag = 1
      if flag == 0:
        temp = cosine_similar_list[j]
        if(temp>max1):
          max1 = temp
          pos1 = j
    max1
    pos1
    top_cosine_pos.append(pos1)
  return top_cosine_pos

top_cosine_pos = topDocuments( cosine_similar_list  )
top_cosine_pos

[1, 0, 0, 0, 0]

In [ ]:
print(top_cosine_pos)
for i in top_cosine_pos:
  print(i)
  print(document_list[i])

[1, 0, 0, 0, 0]
1
[5, 6, 7, 8]
0
[1, 2, 3, 4]
0
[1, 2, 3, 4]
0
[1, 2, 3, 4]
0
[1, 2, 3, 4]
